# 02 Batch profiles / バッチ処理

Process a folder of GLBs (synthetic in this demo), build individual profiles, and overlay them for comparisons.

## Approach / 手法概要
- **Cylinder fitting**: `pyransac3d.Cylinder.fit` (RANSAC) is used to robustly recover the tire axis even when GLB meshes contain noise or outliers. 円筒フィットには外れ値に強い `pyransac3d` のRANSAC推定器を使用します。
- **Coordinate transform**: `align_points` translates the cloud to the fitted axis point, aligns the axis with +X, and rotates the 12 o'clock point to +Z so all downstream stats live in the axial frame. `align_points` で原点を軸上に移動し、軸を +X、12時方向を +Z に揃えてから解析します。
- **Rim zero / リム基準**: rim-line picks regress Z0(Y) so each profile reports Z' = Z − Z0(Y), letting different tires share a common baseline. リムラインを線形回帰して Z0(Y) を算出し、Z' = Z − Z0(Y) を比較指標とします。


In [ ]:
# Ensure the repository root is importable (VS Code / Windows friendly)
import sys
from pathlib import Path

NOTEBOOK_ROOT = Path().resolve()
repo_root = None
for candidate in [NOTEBOOK_ROOT, NOTEBOOK_ROOT.parent, NOTEBOOK_ROOT.parent.parent]:
    if (candidate / "tire_profiler").exists():
        repo_root = candidate
        break

if repo_root is None:
    raise RuntimeError("Unable to locate tire_profiler package relative to this notebook")

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tire_profiler.align import align_points
from tire_profiler.cylinder import fit_cylinder
from tire_profiler.rimline import fit_rimline
from tire_profiler.slice_profile import cylindrical_features, slice_band, compute_profile

In [ ]:
import plotly.graph_objects as go

def preview_cloud(points, color=None, title="Point cloud preview", sample=20000):
    points = np.asarray(points)
    if points.size == 0:
        raise ValueError("No points available for visualization")
    idx = None
    if len(points) > sample:
        rng = np.random.default_rng(0)
        idx = rng.choice(len(points), sample, replace=False)
    pts = points if idx is None else points[idx]
    if color is None:
        color_vals = pts[:, 2]
    else:
        color = np.asarray(color)
        color_vals = color if idx is None else color[idx]
    fig = go.Figure(data=[go.Scatter3d(
        x=pts[:, 0],
        y=pts[:, 1],
        z=pts[:, 2],
        mode='markers',
        marker=dict(size=2, color=color_vals, colorscale='Viridis', opacity=0.85)
    )])
    fig.update_layout(
        title=title,
        scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z', aspectmode='data'),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.show()


In [ ]:
def synthetic_tire_points(seed: int, offset: float = 0.0, n: int = 80000):
    rng = np.random.default_rng(seed)
    x = rng.uniform(-0.12, 0.12, n) + offset
    theta = rng.uniform(-np.pi, np.pi, n)
    radius = 0.34 + 0.003 * np.sin(4 * theta + offset)
    y = radius * np.sin(theta)
    z = radius * np.cos(theta)
    rim_mask = np.abs(x - offset) > 0.09
    z[rim_mask] += 0.004 * np.exp(-((theta[rim_mask]) ** 2) / 0.08)
    pts = np.stack([x, y, z], axis=1) + rng.normal(scale=0.0007, size=(n, 3))
    return pts

In [ ]:
sample_points = synthetic_tire_points(seed=0)
preview_cloud(
    sample_points,
    title="Sample raw point cloud for batch run / バッチ処理前の点群"
)


In [ ]:
def run_profile(seed: int) -> pd.DataFrame:
    pts = synthetic_tire_points(seed, offset=0.0)
    model = fit_cylinder(pts, threshold=0.0025)
    aligned, *_ = align_points(pts, model.point_on_axis, model.axis_direction)
    features = cylindrical_features(aligned, model.radius)
    mask = slice_band(aligned, features=features, tape_width=0.02, outer_band=0.05)
    rim_mask = (np.abs(aligned[:, 0]) > 0.11) & (np.abs(features['arc']) < 0.03)
    rim_points = aligned[rim_mask][:20]
    arc = np.arctan2(rim_points[:, 1], rim_points[:, 2]) * model.radius
    rimline = fit_rimline(rim_points, arc)
    result = compute_profile(aligned, features=features, mask=mask, rimline=rimline, nbins=80)
    return result.profile

In [ ]:
sample_model = fit_cylinder(sample_points, threshold=0.0025)
sample_aligned, *_ = align_points(sample_points, sample_model.point_on_axis, sample_model.axis_direction)
sample_features = cylindrical_features(sample_aligned, sample_model.radius)
sample_mask = slice_band(sample_aligned, features=sample_features, tape_width=0.02, outer_band=0.05)
preview_cloud(
    sample_aligned,
    color=np.where(sample_mask, 1.0, 0.0),
    title="Aligned sample with 12 o'clock mask / 整列後の帯域確認"
)


In [ ]:
profiles = []
for idx, seed in enumerate([0, 1, 2]):
    df = run_profile(seed)
    df = df.assign(tire=f"Tire {idx+1}")
    profiles.append(df)
combined = pd.concat(profiles, ignore_index=True)
combined.head()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
for tire, grp in combined.groupby('tire'):
    ax.plot(grp['x_center'], grp['z_mean'], label=tire)
ax.set_xlabel('X (axial)')
ax.set_ylabel("Z' (rim zero)")
ax.legend()
ax.set_title('Overlay of tire profiles')
ax.grid(True, alpha=0.3)
plt.show()

In [ ]:
stacked = combined.groupby(['tire', 'x_center']).agg(z_mean=('z_mean', 'mean')).reset_index()
agg = stacked.groupby('x_center').agg(mean=('z_mean', 'mean'), std=('z_mean', 'std'))
agg.head()